In [ ]:
# run the network
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# number of iterations in each epoch
n_iter = int(np.rint((len(x_train)-delay)//batch_size)) # round to nearest integer
n_iter_test = int(np.rint((len(x_test)-delay)//batch_size))
n_iter_validation = int(np.rint((len(x_validation)-delay)//batch_size))

# function for generating the time-series batches
def create_batch(x_data,y_data,batch_size,delay,input_size,index):
    
    x_batch = np.zeros([batch_size,delay,input_size])
    y_batch = np.zeros([batch_size,num_classes])
    
    for i in range(batch_size):
        x_batch[i,:,:] = np.reshape(x_data[index*batch_size+i:index*batch_size+i+delay],(delay,num_classes))
        y_batch[i,:] = np.reshape(y_data[index*batch_size+delay+i+1],(1,num_classes))
        
    return x_batch, y_batch

def run_model(inputs,labels,n_iter,train=False,prediction_val):
    for i in range(n_iter):
        # training batch
        x_batch, y_batch = create_batch(inputs,labels,batch_size,delay,input_size,i)

        feed_dict = {x: x_batch, y: y_batch}
        
        # train the net on the data
        if train:
            session.run(optimizer,feed_dict=feed_dict) # run the optimization on the current batch

        loss_val, prediction_val = session.run((loss, prediction), feed_dict=feed_dict)

        # on the last epoch
        if epoch == n_epochs - 1:       
            # save the batch predictions to a list
            prediction_list.extend(prediction_val)    
    

# list for prediction results
prediction_list = []
prediction_list_test = []
prediction_list_validation = []

with tf.Session() as session:
    session.run(init) # initialize variables
    for epoch in range(n_epochs):
        # TRAINING
        for i in range(n_iter):
            # training batch
            x_batch, y_batch = create_batch(x_train,y_train,batch_size,delay,input_size,i)

            feed_dict = {x: x_batch, y: y_batch}
            session.run(optimizer,feed_dict=feed_dict) # run the optimization on the current batch

            loss_val, prediction_val = session.run((loss, prediction), feed_dict=feed_dict)

            # on the last epoch
            if epoch == n_epochs - 1:       
                # save the batch predictions to a list
                prediction_list.extend(prediction_val)
        
        # TESTING
        for i in range(n_iter_test):
            # testing batch
            x_batch, y_batch = create_batch(x_test,y_test,batch_size,delay,input_size,i)
            
            # get the loss and predictions for current testing batch
            feed_dict = {x: x_batch, y: y_batch}
            loss_val_test, prediction_val_test = session.run((loss, prediction), feed_dict=feed_dict)
            
            # on the last epoch
            if epoch == n_epochs - 1:       
                # save the batch predictions to a list
                prediction_list_test.extend(prediction_val_test)            
            
        # increment global step for decaying learning rate at each epoch
        step = session.run(increment_global_step_op)

        # Printing the results at every 1000 epochs
        if epoch % (n_epochs//10) == 0:
            print("Epoch: {}".format(epoch))
            print("TRAINING:")
            print("Loss: {}".format(loss_val))
            print("TEST:")
            print("Loss: {}".format(loss_val_test))
            print("________________________________")
        
    # run the trained net on VALIDATION time-series
    for i in range(n_iter_validation):
        # validation batch
        x_batch, y_batch = create_batch(x_validation,y_validation,batch_size,delay,input_size,i)

        # get the loss and predictions for the validation batch
        feed_dict = {x: x_batch, y: y_batch}
        loss_val_validation, prediction_val_validation = session.run((loss, prediction), feed_dict=feed_dict)

        # on the last epoch
        if epoch == n_epochs - 1:       
            # save the batch predictions to a list
            prediction_list_validation.extend(prediction_val_validation)
            
    print("VALIDATION:")
    print("Loss: {}".format(loss_val_validation))
    
    # save the trained net and variables for later use
    saver.save(session, './checkpoints/LSTMforPredictingLabeFlow')